In [1]:
# /**
# *                   _oo0oo_
# *                  o8888888o
# *                  88" . "88
# *                  (| -_- |)
# *                  0\  =  /0
# *                ___/`---'\___
# *              .' \\|     |// '.
# *             / \\|||  :  |||// \
# *            / _||||| -:- |||||- \
# *           |   | \\\  -  /// |   |
# *           | \_|  ''\---/''  |_/ |
# *           \  .-\__  '-'  ___/-. /
# *         ___'. .'  /--.--\  `. .'___
# *      ."" '<  `.___\_<|>_/___.' >' "".
# *     | | :  `- \`.;`\ _ /`;.`/ - ` : | |
# *     \  \ `_.   \_ __\ /__ _/   .-` /  /
# * =====`-.____`.___ \_____/___.-`___.-'=====
# *                   `=---='
# *
# *
# * ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# *
# *   Buddha blesses your code to be bug free
# */

In [2]:
from pyspark.sql import SparkSession

In [3]:
import pandas as pd
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pyspark.sql.types as T
from pyspark.sql.types import IntegerType
from pyspark.sql.window import Window
from helpers import createDFFromFileAndSchema, clean_special_letters, clean_special_character

In [82]:
import glob
import shutil

In [4]:
spark=SparkSession.builder.appName('Clean up the data and perform the queries').getOrCreate()

In [5]:
spark

In [6]:
SCHEMAS_FOLDER = './schemas/'
FILES_FOLDER = './assets/parsedData/'

# Load and clean Paper DF

In [7]:
### Load paper csv into schema
paper_df = createDFFromFileAndSchema(spark, f'{FILES_FOLDER}papers.csv', f'{SCHEMAS_FOLDER}paper.csv')
paper_df.show()

File path: ./assets/parsedData/papers.csv, schema path: ./schemas/paper.csv
Types from schema: [('paper_id', 'Integer'), ('title', 'String'), ('year', 'Integer')]
+--------+--------------------+----+
|paper_id|               title|year|
+--------+--------------------+----+
|      65|Direct file organ...|1984|
|     130|An introduction t...|1983|
|     195|On solving almost...|1984|
|     260|Connections betwe...|1984|
|     325|Computers and pen...|1984|
|     390|Relativizations c...|1984|
|     455|On the optimum ch...|1984|
|     520|All points addres...|1984|
|     585|Optimum Head Sepa...|1984|
|     650|A parallel-design...|1984|
|     715|Computer - IEEE C...|1984|
|     780|Experience with G...|1984|
|     845|Code generation a...|1984|
|     910|On estimating acc...|1984|
|     975|A distributed alt...|1985|
|    1040|A comparison of t...|1984|
|    1105|Generalizing spec...|1985|
|    1170|Real time graphic...|1984|
|    1235|Common and uncomm...|1984|
|    1300|Foundations o

### Data cleaning for paper schema

In [8]:
### remove spaces from values of the columns
paper_df = paper_df.withColumn("title", trim(paper_df.title))

In [9]:
### check for the correct data types
paper_df.printSchema()

root
 |-- paper_id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- year: integer (nullable = true)



In [9]:
### check for nonsense null data
null_values_paper_df = paper_df.select(
    [count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in paper_df.columns]
)

In [ ]:
null_values_paper_df.show()

In [10]:
### after checking the below dataframes,
### we have seen that all papers, whose title is missing, have the authors (besides paper_id = 748056)
### Decision: fill missing titles with: "Missing Title"

paper_df=paper_df.na.fill('Missing Title', ['title'])

In [11]:
### remove special characters
paper_df=clean_special_character(paper_df,'title')

In [13]:
### check if there are duplicate rows
paper_df.join(
    paper_df
        .groupBy(paper_df.columns) \
        .agg((F.count("*")>1) \
        .cast("int") \
        .alias("Duplicate_indicator")), \
        on=paper_df.columns,how="inner") \
    .show()
### from the dataframe view, we can see that there are no duplicates
paper_df.groupby(['paper_id']).count().where('count > 1').sort('count', ascending=False).show()

+--------+--------------------+----+-------------------+
|paper_id|               title|year|Duplicate_indicator|
+--------+--------------------+----+-------------------+
|      22|On two more Eigen...|1984|                  0|
|      27|Frame theory and ...|1984|                  0|
|      30|Stationary wave s...|1984|                  0|
|      32|Proc. IFIP workin...|1983|                  0|
|      44|ADA Concurrent Pr...|1984|                  0|
|      52|Automated microco...|1984|                  0|
|      58|The application o...|1983|                  0|
|      61|The DISS methodol...|1984|                  0|
|      66|Soft evaluation o...|1984|                  0|
|      91|Design of optimal...|1982|                  0|
|     107|Deterministic pro...|1984|                  0|
|     113|From logic to com...|1983|                  0|
|     114|A first course in...|1983|                  0|
|     122|Proc. of the symp...|1983|                  0|
|     126|Evaluation of ari...|

+--------+-----+
|paper_id|count|
+--------+-----+
+--------+-----+



In [14]:
### check for the number of paper entries
paper_df.count()

2092356

# Load and clean paper_authors

In [12]:
### load paper_authors csv into schema
paper_author_df = createDFFromFileAndSchema(spark, f'{FILES_FOLDER}paper_authors.csv', f'{SCHEMAS_FOLDER}paper_authors.csv')
paper_author_df.show()

File path: ./assets/parsedData/paper_authors.csv, schema path: ./schemas/paper_authors.csv
Types from schema: [('authors', 'String'), ('paper_id', 'Integer')]
+--------------------+--------+
|             authors|paper_id|
+--------------------+--------+
| K Devine;F J. Smith|      65|
|J Wolff von Guden...|     130|
|J. K. Reid;A. Jen...|     195|
|William G. Golson...|     260|
|    Stein Schjolberg|     325|
|W Ian Gasarch;Ste...|     390|
|Sam Toueg;Özalp B...|     455|
|Frederick H. Dill...|     520|
|A. R. Calderbank;...|     585|
|         Uzi Vishkin|     650|
|      Stephen S. Yau|     715|
|Michael D. Schroe...|     780|
|         S L. Graham|     845|
|D Maio;M R. Scala...|     910|
|         Pamela Zave|     975|
|G. Salton;E. Voor...|    1040|
|Douglas D. Dunlop...|    1105|
|Patrick Peruch;Vi...|    1170|
| Robert J. Sternberg|    1235|
|Curtis Roads;John...|    1300|
+--------------------+--------+
only showing top 20 rows



In [13]:
### remove leadind and trailing spaces
paper_author_df = paper_author_df.withColumn("authors", trim(paper_author_df.authors))

In [17]:
### verify schema
paper_author_df.printSchema()

root
 |-- authors: string (nullable = true)
 |-- paper_id: integer (nullable = true)



In [14]:
### remove special letters
paper_author_df=clean_special_letters(paper_author_df, 'authors')

In [19]:
##### DELETE - ?
### check if authors are missing as well for the ids whose title was missing in paper_df
### for rows in paper_author_df.select("authors","paper_id").collect():
###    if rows[1] in null_paper_ids_list:
###        print(rows[0], rows[1])

In [15]:
### split authors so we can have clean data and separate records { paper_id; author }
unique_paper_author_df = paper_author_df \
    .select(F.col("paper_id"), F.explode(F.split(F.col("authors"),";")).alias("author"))

unique_paper_author_df.show(20, False)

+--------+---------------------+
|paper_id|author               |
+--------+---------------------+
|65      |K Devine             |
|65      |F J. Smith           |
|130     |J Wolff von Gudenberg|
|195     |J. K. Reid           |
|195     |A. Jennings          |
|260     |William G. Golson    |
|260     |William C. Rounds    |
|325     |Stein Schjolberg     |
|390     |W Ian Gasarch        |
|390     |Steven Homer         |
|455     |Sam Toueg            |
|455     |zalp Babaoğlu        |
|520     |Frederick H. Dill    |
|520     |Satish Gupta         |
|520     |Daniel T. Ling       |
|520     |Richard E. Matick    |
|585     |A. R. Calderbank     |
|585     |E. G. Coffman, Jr.   |
|585     |L. Flatto            |
|650     |Uzi Vishkin          |
+--------+---------------------+
only showing top 20 rows



In [16]:
### remove leadind and trailing spaces
unique_paper_author_df = unique_paper_author_df.withColumn("author", trim(unique_paper_author_df.author))

In [22]:
unique_paper_author_df.printSchema()

root
 |-- paper_id: integer (nullable = true)
 |-- author: string (nullable = true)



In [17]:
### remove special characters
unique_paper_author_df=clean_special_character(unique_paper_author_df, 'author')

In [18]:
### lowercase author-name
unique_paper_author_df=unique_paper_author_df.withColumn('author', lower(col('author')))

In [19]:
### check for duplicate rows:
unique_paper_author_df \
    .groupby(['paper_id', 'author']) \
    .count() \
    .where('count > 1') \
    .sort('count', ascending=False) \
    .show()

### drop duplicate rows since here we need unique paper-author relation
unique_paper_author_df = unique_paper_author_df.dropDuplicates()

22/01/21 14:16:26 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/01/21 14:16:26 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/01/21 14:16:27 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/01/21 14:16:27 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/01/21 14:16:27 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/01/21 14:16:27 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/01/21 14:16:27 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/01/21 14:16:38 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/01/21 14:16:38 WARN RowBasedKeyValueBatch: Calling spill() on

+--------+-----------------+-----+
|paper_id|           author|count|
+--------+-----------------+-----+
| 1523221|     dongkun shin|    4|
| 2059316|    han chuanfeng|    3|
| 2040206|     mingshu wang|    3|
| 1202294|        n. sharma|    3|
| 2042230|          lu leng|    3|
| 2040206|     anchun cheng|    3|
| 1070921|     joseph turow|    2|
|  384591| simon st laurent|    2|
| 1936484|    jiangxia duan|    2|
| 1070928|  federico rajola|    2|
| 1778146|  luis m. vaquero|    2|
| 1867844|m. h. chehreghani|    2|
| 1612982|       a. mammoli|    2|
| 1070951| alexander horsch|    2|
|  393283|  jeffrey richter|    2|
| 1058622|  yehuda e. kalay|    2|
| 1065597|       philip bos|    2|
| 1069185|    danica kragic|    2|
| 1069190|    mick kerrigan|    2|
| 1070401|    arieh iserles|    2|
+--------+-----------------+-----+
only showing top 20 rows



In [20]:
### split author-name so we keep only the surname
### This decision was made because there are a lot of disrepancies in full names,
### and later on we need the clean and nice surnames for correct joining of the tables by paper_id + surname.
unique_paper_author_cleaned_df = unique_paper_author_df \
    .select(F.col("paper_id"), F.trim(F.element_at (F.split(F.col("author")," "),-1)).alias('name'))

unique_paper_author_cleaned_df.show()

+--------+------------+
|paper_id|        name|
+--------+------------+
|    1821|        suri|
|    3582|    williams|
|    8388|        huet|
|    8581|    stoddart|
|   11188|        maio|
|   11703|      landau|
|   11831|      reeves|
|   13267|     searles|
|   16580|      mettke|
|   16965|schuitemaker|
|   18077|      buxton|
|   18141|  hemmerling|
|   18208|  linebarger|
|   18530|      millen|
|   19959|         jr.|
|   20152|      soupos|
|   25486|      mazaud|
|   25614|      mccain|
|   26522| jagannathan|
|   26783|       sheil|
+--------+------------+
only showing top 20 rows



In [ ]:
### check how many entries there are in unique_paper_author_cleaned_df
unique_paper_author_cleaned_df.count()

# Clean and Load Authors df

In [22]:
### load author csv into schema
author_df = createDFFromFileAndSchema(spark, f'{FILES_FOLDER}authors.csv', f'{SCHEMAS_FOLDER}author.csv')
author_df.show()

File path: ./assets/parsedData/authors.csv, schema path: ./schemas/author.csv
Types from schema: [('author_id', 'Integer'), ('citation_count', 'Integer'), ('h_index', 'Integer'), ('name', 'String'), ('paper_count', 'Integer')]
+---------+--------------+-------+--------------------+-----------+
|author_id|citation_count|h_index|                name|paper_count|
+---------+--------------+-------+--------------------+-----------+
|       17|             0|      0|     J. Michael Howe|          1|
|       34|             0|      0|        Haitham Gabr|          2|
|       51|             4|      1|         Emma Tonkin|          8|
|       68|             1|      1|        Woochul Shin|          4|
|       85|             0|      0|           S Improta|          1|
|      102|             8|      2|       Richard Ferri|          5|
|      119|             0|      0|            Qing Liu|          1|
|      136|             0|      0|      Artur Gramacki|          2|
|      153|             0

In [23]:
### remove spaces from values of the column
author_df = author_df.withColumn("name", trim(author_df.name))

In [30]:
### check for nonsense null data (filtering will be done later)
null_values_author_df = author_df \
    .select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in author_df.columns])
null_values_author_df.show()

+---------+--------------+-------+----+-----------+
|author_id|citation_count|h_index|name|paper_count|
+---------+--------------+-------+----+-----------+
|        0|             2|      2|   3|          2|
+---------+--------------+-------+----+-----------+



In [24]:
### fill empty paper_count, citation_count, h_index to 0 (just one author)
author_df=author_df.na.fill(value=0, subset='paper_count')
author_df=author_df.na.fill(value=0, subset='citation_count')
author_df=author_df.na.fill(value=0, subset='h_index')

In [25]:
### remove special characters like í, â, é
author_df=clean_special_letters(author_df, 'name')

In [26]:
### remove special characters
author_df=clean_special_character(author_df,'name')

In [27]:
### lowercase author-name
author_df=author_df.withColumn('name', lower(col('name')))

In [36]:
### check if there are duplicate author_ids
author_df.groupby(['author_id']).count().where('count > 1').sort('count', ascending=False).show()

+---------+-----+
|author_id|count|
+---------+-----+
+---------+-----+



# Join author and paper dataframes to ensure consistency

Load and clean Author2Paper (from the supplement txt file)

In [28]:
### load paper_author_id csv into schema
dtypes = pd.read_csv('./schemas/paper_author_id.csv').to_records(index=False).tolist()
print(dtypes)
fields = [T.StructField(dtype[0], globals()[f'{dtype[1]}Type']()) for dtype in dtypes]
schema = StructType(fields)
author_id_2_paper_id_df = spark.read.options(delimiter='\t').csv('./assets/AMiner-Author2Paper.txt', header=False,schema=schema)

[('index', 'Integer'), ('author_id', 'Integer'), ('paper_id', 'Integer'), ('author_position', 'Integer')]


In [29]:
### check how many entries there are
author_id_2_paper_id_df.count()

5192998

In [30]:
### check for duplicates between paper_id and author_id
author_id_2_paper_id_df \
    .groupby(['author_id', 'paper_id']) \
    .count().where('count > 1') \
    .sort('count', ascending=False).show(10,False)


22/01/21 14:18:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/01/21 14:18:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/01/21 14:18:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/01/21 14:18:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/01/21 14:18:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/01/21 14:18:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/01/21 14:18:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/01/21 14:18:56 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/01/21 14:18:56 WARN RowBasedKeyValueBatch: Calling spill() on

+---------+--------+-----+
|author_id|paper_id|count|
+---------+--------+-----+
+---------+--------+-----+



In [31]:
### join the suplementary author_id_2_paper_id_df df with author_df
author_id_2_paper_id_extended_df = author_id_2_paper_id_df.join(author_df, 'author_id', 'left').drop('index')

In [32]:
### split the author-name to keep only the surname ---> cleaning data
author_id_2_paper_id_cleaned_df = author_id_2_paper_id_extended_df \
    .select( \
        F.col("paper_id"), \
        F.col("author_id"), \
        F.col("citation_count"), \
        F.col("h_index"), \
        F.col("paper_count"), \
        F.trim(F.element_at(F.split(F.col("name")," "),-1)).alias('name') \
    )

In [33]:
### join the cleaned paper_2_author_cleaned_df to unique_paper_author_cleaned_df
final_paper_author_id_df = unique_paper_author_cleaned_df \
    .join(author_id_2_paper_id_cleaned_df, ['name', 'paper_id'], 'inner') \
    .dropDuplicates()

In [43]:
### check how many entries there are in the final dataframe
final_paper_author_id_df.count()

[292.095s][warning][gc,alloc] Executor task launch worker for task 2.0 in stage 55.0 (TID 194): Retried waiting for GCLocker too often allocating 1048578 words


22/01/21 13:54:32 WARN TaskMemoryManager: Failed to allocate a page (8388608 bytes), try again.


5114352

In [ ]:
final_paper_author_id_df.show(100)

# Load and clean Affiliations df

In [34]:
### load affiliation csv into schema
affiliation_df = createDFFromFileAndSchema(spark, f'{FILES_FOLDER}affiliations.csv', f'{SCHEMAS_FOLDER}affiliation.csv')
affiliation_df.show()

File path: ./assets/parsedData/affiliations.csv, schema path: ./schemas/affiliation.csv
Types from schema: [('affiliations', 'String'), ('paper_id', 'Integer')]
+--------------------+--------+
|        affiliations|paper_id|
+--------------------+--------+
|The Queen's Unive...|      65|
|Univ. of Karlsruh...|     130|
|AERE Harwell Labo...|     195|
|University of Mic...|     260|
|Oslo politikammer...|     325|
|Harvard Univ., Ca...|     390|
|Cornell Univ., It...|     455|
|IBM General Techn...|     520|
|               -;-;-|     585|
|New York Univ., N...|     650|
|                   -|     715|
|Xerox Palo Alto R...|     780|
|Univ. of Californ...|     845|
|University of Bol...|     910|
|AT & T Bell Labor...|     975|
|Cornell Univ., It...|    1040|
|University of Mar...|    1105|
|Laboratoire de Ps...|    1170|
|Yale Univ., New H...|    1235|
|                 -;-|    1300|
+--------------------+--------+
only showing top 20 rows



In [35]:
### remove leading and trailing spaces
affiliation_df = affiliation_df.withColumn("affiliations", trim(affiliation_df.affiliations))

affiliation_df.printSchema()
affiliation_df.show()

root
 |-- affiliations: string (nullable = true)
 |-- paper_id: integer (nullable = true)

+--------------------+--------+
|        affiliations|paper_id|
+--------------------+--------+
|The Queen's Unive...|      65|
|Univ. of Karlsruh...|     130|
|AERE Harwell Labo...|     195|
|University of Mic...|     260|
|Oslo politikammer...|     325|
|Harvard Univ., Ca...|     390|
|Cornell Univ., It...|     455|
|IBM General Techn...|     520|
|               -;-;-|     585|
|New York Univ., N...|     650|
|                   -|     715|
|Xerox Palo Alto R...|     780|
|Univ. of Californ...|     845|
|University of Bol...|     910|
|AT & T Bell Labor...|     975|
|Cornell Univ., It...|    1040|
|University of Mar...|    1105|
|Laboratoire de Ps...|    1170|
|Yale Univ., New H...|    1235|
|                 -;-|    1300|
+--------------------+--------+
only showing top 20 rows



In [36]:
### check for null values in the affiliations column
### we can see, that there are many rows with null affiliations
### these rows will be cleaned up further
null_values_affiliations=affiliation_df \
    .select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in affiliation_df.columns])
print(null_values_affiliations.show())

+------------+--------+
|affiliations|paper_id|
+------------+--------+
|       37499|       0|
+------------+--------+

None


In [37]:
### This df will in the end be used to count papers per unique affiliation,
### so if the affiliation is missing, it doesnt make sense to keep the row
### Decision: drop all rows where affiliation is null
affiliation_df=affiliation_df.na.drop(how="any", subset=['affiliations'])

In [38]:
### split affiliations so we can have clean data and separate records { paper_id; affiliation }
unique_affiliations_df = affiliation_df \
    .select(F.col("paper_id"), F.explode(F.split(F.col("affiliations"),";")) \
    .alias("affiliation"))
unique_affiliations_df.show(20, False)

+--------+------------------------------------------------------+
|paper_id|affiliation                                           |
+--------+------------------------------------------------------+
|65      |The Queen's University of Belfast, Belfast, UK        |
|65      |The Queen's University of Belfast, Belfast, UK        |
|130     |Univ. of Karlsruhe, Karlsruhe, West Germany           |
|195     |AERE Harwell Laboratory, Oxon, UK                     |
|195     |Queen's Univ., Belfast, Northern Ireland              |
|260     |University of Michigan, Ann Arbor, MI                 |
|260     |University of Michigan, Ann Arbor, MI                 |
|325     |Oslo politikammer, Oslo, Norway                       |
|390     |Harvard Univ., Cambridge, MA                          |
|390     |Boston Univ., Boston, MA                              |
|455     |Cornell Univ., Ithaca, NY                             |
|455     |Cornell Univ., Ithaca, NY                             |
|520     |

In [39]:
### check for special nonsense characters "-" and filter them out
### If the affiliation is missing, there is no point of keeping the rows
unique_affiliations_df = unique_affiliations_df.where(unique_affiliations_df.affiliation != '-')

In [ ]:
unique_affiliations_df.show()

In [ ]:
### check for duplicate rows
unique_affiliations_df \
    .groupby(['paper_id', 'affiliation']) \
    .count() \
    .where('count > 1') \
    .sort('count', ascending=False) \
    .show()

In [40]:
### drop duplicate rows since here we need unique affiliations
unique_affiliations_df = unique_affiliations_df.dropDuplicates()

In [41]:
### remove special characters
unique_affiliations_df = clean_special_character(unique_affiliations_df, 'affiliation')
### remove special letters
unique_affiliations_df = clean_special_letters(unique_affiliations_df, 'affiliation')

In [42]:
### lowercase the affiliation col values
unique_affiliations_df = unique_affiliations_df.withColumn('affiliation', lower(col('affiliation')))

In [ ]:
### check for duplicate rows
unique_affiliations_df.groupby(['paper_id', 'affiliation']) \
    .count() \
    .where('count > 1') \
    .sort('count', ascending=False) \
    .show(truncate=False)

In [43]:
### drop duplicates
unique_affiliations_df = unique_affiliations_df.dropDuplicates()

In [44]:
### filter out affiliations that are not part of the final_paper_author_id_df
final_affiliations_df = unique_affiliations_df \
    .join(final_paper_author_id_df, ['paper_id'], 'inner') \
    .select(F.col('paper_id'), F.col('affiliation')) \
    .dropDuplicates()

# Load and clean Publication_venues df

In [45]:
### load publication_venues into schema
publication_venue_df = createDFFromFileAndSchema( \
    spark, f'{FILES_FOLDER}publication_venues.csv', f'{SCHEMAS_FOLDER}publication_venues.csv' \
)

File path: ./assets/parsedData/publication_venues.csv, schema path: ./schemas/publication_venues.csv
Types from schema: [('paper_id', 'Integer'), ('publication_venue', 'String')]


In [46]:
### trim publication_venue
publication_venue_df = publication_venue_df.withColumn( \
    "publication_venue", trim(publication_venue_df.publication_venue) \
)
publication_venue_df.show()

+--------+--------------------+
|paper_id|   publication_venue|
+--------+--------------------+
|      65|Information Techn...|
|     130|Proc. of the symp...|
|     195|ACM Transactions ...|
|     260|Information and C...|
|     325|Computers and pen...|
|     390|Information and C...|
|     455|SIAM Journal on C...|
|     520|IBM Journal of Re...|
|     585|Journal of the AC...|
|     650|Theoretical Compu...|
|     715|            Computer|
|     780|ACM Transactions ...|
|     845|Methods and tools...|
|     910|Information Proce...|
|     975|ACM Transactions ...|
|    1040|Information Proce...|
|    1105|ACM Transactions ...|
|    1170|Proc. of the 2nd ...|
|    1235|Proc. of the inte...|
|    1300|Foundations of co...|
+--------+--------------------+
only showing top 20 rows



In [47]:
### check for null values inside publication venues dataframe
null_values_publication_venue = publication_venue_df \
    .select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in publication_venue_df.columns])

In [48]:
null_values_publication_venue.show()

+--------+-----------------+
|paper_id|publication_venue|
+--------+-----------------+
|       0|              148|
+--------+-----------------+



In [49]:
### drop null values 
publication_venue_df=publication_venue_df.na.drop(how="any", subset=['publication_venue'])

In [ ]:
### check for duplicate rows (no duplicates found)
publication_venue_df \
    .groupby(['paper_id', 'publication_venue']) \
    .count() \
    .where('count > 1').sort('count', ascending=False).show()

In [50]:
### join publication_venue_df with final_paper_author_id_df
### to remove publication venues of paper ids which are not part of final_paper_author_id_df
final_publication_venues_df = publication_venue_df \
    .join(final_paper_author_id_df, ['paper_id'], 'inner') \
    .select(F.col('paper_id'), F.col('publication_venue')).dropDuplicates()

# Load and clean Citations df

In [51]:
### load affiliation csv into schema
citation_df = createDFFromFileAndSchema(spark, f'{FILES_FOLDER}citations.csv', f'{SCHEMAS_FOLDER}citations.csv')
citation_df.show()

File path: ./assets/parsedData/citations.csv, schema path: ./schemas/citations.csv
Types from schema: [('paper_id', 'Integer'), ('ref_ids', 'String')]
+--------+--------------------+
|paper_id|             ref_ids|
+--------+--------------------+
|      65|                null|
|     130|                null|
|     195|317424;317425;317573|
|     260|                null|
|     325|                null|
|     390|                null|
|     455|                null|
|     520|       318368;323493|
|     585|                null|
|     650|                null|
|     715|                null|
|     780|318420;319233;319...|
|     845|                null|
|     910|                null|
|     975|67604;318882;3718...|
|    1040|                null|
|    1105|289087;318014;318...|
|    1170|                null|
|    1235|                null|
|    1300|                null|
+--------+--------------------+
only showing top 20 rows



In [52]:
### remove leading and trailing spaces
citation_df = citation_df.withColumn("ref_ids", trim(citation_df.ref_ids))

In [ ]:
### show how many paper ids are present in citation_df
citation_df.select(countDistinct('paper_id')).show()

In [ ]:
### check for duplicate rows (no duplicates found)
citation_df.groupby(['paper_id', 'ref_ids']).count().where('count > 1').sort('count', ascending=False).show()

In [53]:
### split citations so we can have clean data and seperate records {paper_id; ref_id}
unique_citation_df = citation_df \
    .select(F.col("paper_id"), F.explode_outer(F.split(F.col("ref_ids"),";")).alias("ref_id"))
unique_citation_df.show(20, False)

+--------+------+
|paper_id|ref_id|
+--------+------+
|65      |null  |
|130     |null  |
|195     |317424|
|195     |317425|
|195     |317573|
|260     |null  |
|325     |null  |
|390     |null  |
|455     |null  |
|520     |318368|
|520     |323493|
|585     |null  |
|650     |null  |
|715     |null  |
|780     |318420|
|780     |319233|
|780     |319290|
|780     |319579|
|780     |320813|
|845     |null  |
+--------+------+
only showing top 20 rows



In [ ]:
### find the number of unique papers in unique_citation_df dataframe
unique_citation_df.select(countDistinct('paper_id')).show()

In [54]:
### remove leading and trailing spaces
unique_citation_df = unique_citation_df.withColumn("ref_id", trim(unique_citation_df.ref_id))
### change data type of ref_id to Integer
unique_citation_df = unique_citation_df.withColumn("ref_id",unique_citation_df["ref_id"].cast(IntegerType()))

In [ ]:
unique_citation_df.printSchema()

In [ ]:
### check for duplicate rows
unique_citation_df.groupby(['paper_id', 'ref_id']).count().where('count > 1').sort('count', ascending=False).show()

In [ ]:
### check whether some papers have null ref_ids
### (those papers will still be kept in the dataframe for consistency)
unique_citation_df.filter(unique_citation_df['ref_id'].isNull()).show()

In [55]:
### clean up ref ids if their paper_id is not a part of final_paper_author_id_df
final_citation_df = unique_citation_df \
    .join(final_paper_author_id_df, ['paper_id'], 'inner') \
    .select(F.col('paper_id'), F.col('ref_id')) \
    .dropDuplicates()

# Load and Research_interests in df

In [56]:
### load research_interests csv into schema
research_interests_df = createDFFromFileAndSchema( \
    spark, f'{FILES_FOLDER}research_interests.csv', f'{SCHEMAS_FOLDER}research_interests.csv' \
)

File path: ./assets/parsedData/research_interests.csv, schema path: ./schemas/research_interests.csv
Types from schema: [('author_id', 'Integer'), ('research_interests', 'String')]


In [57]:
### remove leading and trailing spaces
research_interests_df = research_interests_df \
    .withColumn("research_interests", trim(research_interests_df.research_interests))

In [67]:
### check for null values in the affiliations column
research_interests_df \
    .select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in research_interests_df.columns]).show()
### drop null values (it is safe since we dont need research_interests for any computation)
research_interests_df = research_interests_df.na.drop(how="any", subset=['research_interests'])

+---------+------------------+
|author_id|research_interests|
+---------+------------------+
|        0|             15398|
+---------+------------------+



In [ ]:
research_interests_df.printSchema()
research_interests_df.show()

In [58]:
### split research interests
### so we can have clean data and separate records { paper_id; research_interest }
unique_research_interests_df = research_interests_df \
    .select(F.col("author_id"), F.explode(F.split(F.col("research_interests"),";")) \
    .alias("research_interest"))

unique_research_interests_df.show(20, False)

+---------+-----------------------------------------+
|author_id|research_interest                        |
+---------+-----------------------------------------+
|17       |HIV disease                              |
|17       |Internet resource                        |
|17       |World-Wide Web                           |
|17       |clinical management                      |
|34       |associate polynomial term                |
|34       |bivariate polynomial                     |
|34       |difficult computational problem          |
|34       |novel polynomial                         |
|34       |polynomial multiplication problem        |
|34       |polynomial term                          |
|34       |reachability problem                     |
|34       |Probabilistic Reachability               |
|34       |Reachability analysis                    |
|34       |better time complexity                   |
|51       |metadata element                         |
|51       |metadata record  

In [59]:
### remove leading and trailing spaces
unique_research_interests_df = unique_research_interests_df \
    .withColumn("research_interest", trim(unique_research_interests_df.research_interest))

In [60]:
#### clean special characters and special letters
unique_research_interests_df = clean_special_character(unique_research_interests_df, 'research_interest')
unique_research_interests_df = clean_special_letters(unique_research_interests_df, 'research_interest')
### lowercase research_interests
unique_research_interests_df = unique_research_interests_df \
    .withColumn('research_interest', lower(col('research_interest')))

In [ ]:
### check for duplicate rows:
unique_research_interests_df \
    .groupby(['author_id', 'research_interest']) \
    .count() \
    .where('count > 1') \
    .sort('count', ascending=False).show()

In [61]:
### drop duplicates
unique_research_interests_df=unique_research_interests_df.dropDuplicates()

In [62]:
# clean up research interests if their author id is not a part of final_paper_author_id_df
final_research_interests_df = unique_research_interests_df \
    .join(final_paper_author_id_df, ['author_id'], 'inner') \
    .select(F.col('author_id'), F.col('research_interest')) \
    .dropDuplicates()

# Run Queries

### Q1.2 Compute paper count per unique affiliation

In [73]:
paper_count_per_affiliation_df = final_affiliations_df \
    .groupBy('affiliation') \
    .count() \
    .withColumnRenamed("count", "papers_count")

In [ ]:
### count the number of results
paper_count_per_affiliation_df.count()

In [ ]:
paper_count_per_affiliation_df.show()

### Q1.1 Validate precomputed paper counts, citation (ref) counts and h-indexes (per author)

#### How to compute h-index for a specific author
1. Retrieve all publications of the author
2. Calculate the number of references per publication
3. Sort the results in descending order
4. Find a threshold N, where N top publications have at least N references each. N is the h-index of the author.


In [66]:
# 1-2 Calculate the number of references per publication
refs_per_paper_count_df = final_citation_df \
    .groupBy("paper_id") \
    .count() \
    .withColumnRenamed("count","paper_references")

In [74]:
refs_per_paper_count_df.count()

2029262

In [67]:
### 3 Join [papers per author] with [references per paper] and sort the results in descending order
author_papers_with_ref_count = final_paper_author_id_df.join(refs_per_paper_count_df, 'paper_id') \
    .sort(col("paper_references").desc())

In [ ]:
author_papers_with_ref_count.show()

In [68]:
### add index column to table to ease h-index calculation
window = Window.partitionBy(author_papers_with_ref_count['author_id']) \
    .orderBy(desc("paper_references"), desc("paper_id"))

indexed_grouped_papers_df = author_papers_with_ref_count.select('*', rank().over(window).alias('index'))


In [69]:
h_indexed_papers = indexed_grouped_papers_df \
    .withColumn("possible_h_index", \
                when( \
                    indexed_grouped_papers_df.index <= indexed_grouped_papers_df.paper_references, \
                    indexed_grouped_papers_df.index \
                    ).otherwise(0) \
                )

In [ ]:
h_indexed_papers.show()

In [70]:
h_indexed_grouped_by_author_papers_df = h_indexed_papers.groupBy('author_id')

In [71]:
h_indexed_aggregated_papers_df = h_indexed_grouped_by_author_papers_df.agg( \
        F.count("paper_id").alias("validated_paper_count"),
        F.sum("paper_references").alias("validated_citation_count"),
        F.max("possible_h_index").alias("validated_h_index")
    )

#### Computed results for `validated_paper_count`, `validated_paper_count` and `validated_h_index`

__Final resulting dataframe is unique_authors_with_validated_cols_df__

In [72]:
### Join the computation results with author dataframe
### to be able to compare received values with the precomputed values
unique_authors_with_validated_cols_df = h_indexed_aggregated_papers_df.join(author_df, 'author_id', 'inner' )

In [ ]:
### count the number of entries
unique_authors_with_validated_cols_df.count()

In [ ]:
unique_authors_with_validated_cols_df.show()

In [ ]:
### For the info: check how many precomputed h-indexes differ from the validated h-indexes
filter_condition = unique_authors_with_validated_cols_df["validated_h_index"] != unique_authors_with_validated_cols_df["h_index"]
unique_authors_with_validated_cols_df \
    .filter(filter_condition) \
    .count()

In [ ]:
### For the info: check how many precomputed author citation counts differ from the validated citation counts
filter_condition = unique_authors_with_validated_cols_df["validated_citation_count"] != unique_authors_with_validated_cols_df["citation_count"]
unique_authors_with_validated_cols_df \
    .filter(filter_condition) \
    .count()

In [ ]:
### For the info: check how many precomputed author paper counts differ from the validated paper counts
filter_condition = unique_authors_with_validated_cols_df["validated_paper_count"] != unique_authors_with_validated_cols_df["paper_count"]
unique_authors_with_validated_cols_df.filter(filter_condition).count()

# Save cleaned & computed data into the csv files

In [102]:
CLEAN_DATA_FOLDER = './assets/cleanedDFsData/'
def saveDFIntoCSVFolder(df, folderName):
    # Save data to csv file
    df.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save(f'{CLEAN_DATA_FOLDER}{folderName}')
def moveFileToCorrectFolder(folderName):
    filename = glob.glob(f'{CLEAN_DATA_FOLDER}{folderName}/*.csv')[0]
    shutil.move(filename, f'{CLEAN_DATA_FOLDER}{folderName}_ds.csv')
    
# After saving of the data to csv we have to call moveFileToCorrectFolder,
# because saveDFIntoCSVFolder actually saves the data into the folder with a csv file inside.
# moveFileToCorrectFolder moves the file to the correct location.

In [ ]:
## Saving cleaned & unique paper author data
saveDFIntoCSVFolder(final_paper_author_id_df, 'paper_author')

In [93]:
moveFileToCorrectFolder('paper_author')

In [94]:
## Saving cleaned & unique affiliations data with computed paper_count
saveDFIntoCSVFolder(paper_count_per_affiliation_df, 'affiliation_paper_count')

22/01/21 15:18:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/01/21 15:18:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/01/21 15:18:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/01/21 15:18:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/01/21 15:18:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/01/21 15:18:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/01/21 15:18:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/01/21 15:18:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


In [95]:
moveFileToCorrectFolder('affiliation_paper_count')

In [104]:
## Saving cleaned & unique affiliations data with computed paper_count
saveDFIntoCSVFolder(final_affiliations_df, 'affiliations')

In [105]:
moveFileToCorrectFolder('affiliations')

In [ ]:
## Saving cleaned & unique publication venues data 
saveDFIntoCSVFolder(final_publication_venues_df, 'publication_venues')

In [107]:
moveFileToCorrectFolder('publication_venues')

In [108]:
## Saving cleaned & unique citations data
saveDFIntoCSVFolder(final_citation_df, 'citations')

In [109]:
moveFileToCorrectFolder('citations')

In [110]:
## Saving cleaned & computed & research interests data
saveDFIntoCSVFolder(final_research_interests_df, 'research_interests')

In [111]:
moveFileToCorrectFolder('research_interests')